Installing necessary packages:
```Julia
using Pkg

Pkg.add(["Random", "Distributions", "DataFrames", "GLM", "Statistics", "ProgressMeter", "Plots"])
```

In [1]:
using Random, Distributions, DataFrames, GLM, Statistics, ProgressMeter, CSV

In [2]:
α = 1.18
β = .4
Q = 20 # Liczba państw
nsims = 10000

N_distr = Poisson(2000)
M_distr = (N, α) -> Poisson.(N .^ α)
p_distr = Uniform(.1,.75)

res = Vector{Any}(missing, nsims)

prog = Progress(10*Threads.nthreads(), "Simulation progress ...")

Threads.@threads for i in 1:(10*Threads.nthreads())
    Threads.threadid()
    next!(prog)
end # end for

Simulation progress ... 100%|████████████████████████████| Time: 0:00:00


In [3]:
prog = Progress(nsims, "Simulation progress ...")

Threads.@threads for k in 1:nsims
    N = reduce(vcat, rand.(Binomial.(rand(N_distr, Q), .1), 1))
    M = reduce(vcat, rand.(M_distr(N, α), 1))
    p = rand(p_distr, Q)
    
    m = reduce(vcat, rand.(Poisson.(M .* p), 1))
    n = reduce(vcat, rand.(Binomial.(N, p), 1))

    df1 = DataFrame(
        y = m,
        x1 = log.(N),
        x2 = log.(n ./ N)
    )

    mm = glm(@formula(y ~ x1 + x2 + 0), df1, Poisson(), LogLink())
    α̂₁, β̂₁ = coef(mm)

    ols = lm(@formula(log(y) ~ x1 + x2 + 0), df1)
    α̂₂, β̂₂ = coef(ols)

    res[k] = [sum(M) sum(N .^ α) sum(N .^ α̂₁) sum(N .^ α̂₂)]
    #push!(res, [sum(M) sum(N .^ α) sum(N .^ α̂₁) sum(N .^ α̂₂)])
    next!(prog)
end # end for

Simulation progress ... 100%|████████████████████████████| Time: 0:00:04


In [4]:
df_res = DataFrame(reduce(vcat, [res[i] for i in 1:length(res) if isassigned(res, i)]), ["actual", "expected", "glm_est", "ols_est"])
df_plot = stack(df_res, 1:4)

describe(df_res)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Int64,DataType
1,actual,10388.0,9603.0,10386.0,11292.0,0,Float64
2,expected,10387.5,9510.32,10388.1,11058.1,0,Float64
3,glm_est,10125.9,7959.86,10113.9,12609.8,0,Float64
4,ols_est,10045.0,7744.76,10032.5,13860.3,0,Float64


In [5]:
DataFrame(
    rel_bias = [mean((df_res[:, 1] .- df_res[:, 2]) ./ df_res[:, 1]), mean((df_res[:, 1] .- df_res[:, 3]) ./ df_res[:, 1]), mean((df_res[:, 1] .- df_res[:, 4]) ./ df_res[:, 1])],
    rel_mse  = [mean(((df_res[:, 1] .- df_res[:, 2]) .^ 2) ./ df_res[:, 1]), mean(((df_res[:, 1] .- df_res[:, 3]) .^ 2) ./ df_res[:, 1]), mean(((df_res[:, 1] .- df_res[:, 4]) .^ 2) ./ df_res[:, 1])],
    rel_mae  = [mean(abs.(df_res[:, 1] .- df_res[:, 2]) ./ df_res[:, 1]), mean(abs.(df_res[:, 1] .- df_res[:, 3]) ./ df_res[:, 1]), mean(abs.(df_res[:, 1] .- df_res[:, 4]) ./ df_res[:, 1])],
    est      = ["Expected value","glm","ols"]
)

Row,rel_bias,rel_mse,rel_mae,est
,Float64,Float64,Float64,String
1,-4.89825e-5,0.991537,0.00781908,Expected value
2,0.0252293,33.4901,0.0456803,glm
3,0.0330229,49.4803,0.0557081,ols


In [6]:
DataFrame(
    rel_bias = [mean((df_res[:, 2] .- df_res[:, 3]) ./ df_res[:, 2]), mean((df_res[:, 2] .- df_res[:, 4]) ./ df_res[:, 2])],
    rel_mse  = [mean(((df_res[:, 2] .- df_res[:, 3]) .^ 2) ./ df_res[:, 2]), mean(((df_res[:, 2] .- df_res[:, 4]) .^ 2) ./ df_res[:, 2])],
    rel_mae  = [mean(abs.(df_res[:, 2] .- df_res[:, 3]) ./ df_res[:, 2]), mean(abs.(df_res[:, 2] .- df_res[:, 4]) ./ df_res[:, 2])],
    est      = ["glm","ols"]
)

Row,rel_bias,rel_mse,rel_mae,est
,Float64,Float64,Float64,String
1,0.0251827,34.4351,0.0463156,glm
2,0.0329732,50.4747,0.0562796,ols


In [7]:
CSV.write(pwd() * "/data_raw/init_res.csv", df_res);